In [ ]:
# %% [markdown]
# # 🎯 CTDE Leader-Follower MAS Event-Triggered Consensus
# 
# **CTDE 架构（Centralized Training Decentralized Execution）**
# - **Actor**: 分散式，只使用本地观测
# - **Critic**: 集中式，使用全局状态 + 联合动作
# - **随机初始化**: 每个 episode 随机化领导者动力学和跟随者初始状态

# %%
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from config import print_config, NUM_FOLLOWERS, NUM_PINNED
print_config()

# %% [markdown]
# ## 🎲 Randomization Settings
# 
# 每个 episode 随机初始化领导者动力学和跟随者状态，提高泛化能力

# %%
from config import (
    RANDOMIZE_LEADER, RANDOMIZE_FOLLOWER,
    LEADER_AMPLITUDE_RANGE, LEADER_OMEGA_RANGE, LEADER_PHASE_RANGE,
    FOLLOWER_INIT_POS_STD_RANGE, FOLLOWER_INIT_VEL_STD_RANGE,
    LEADER_AMPLITUDE, LEADER_OMEGA, LEADER_PHASE
)

print("🎲 Randomization Configuration:")
print("=" * 50)
print(f"\n📍 Leader Dynamics Randomization: {'✅ Enabled' if RANDOMIZE_LEADER else '❌ Disabled'}")
if RANDOMIZE_LEADER:
    print(f"   Amplitude range: {LEADER_AMPLITUDE_RANGE} (base: {LEADER_AMPLITUDE})")
    print(f"   Omega range: {LEADER_OMEGA_RANGE} (base: {LEADER_OMEGA})")
    print(f"   Phase range: [{LEADER_PHASE_RANGE[0]:.2f}, {LEADER_PHASE_RANGE[1]:.2f}] rad")
else:
    print(f"   Fixed amplitude: {LEADER_AMPLITUDE}")
    print(f"   Fixed omega: {LEADER_OMEGA}")
    print(f"   Fixed phase: {LEADER_PHASE}")

print(f"\n📍 Follower Init Randomization: {'✅ Enabled' if RANDOMIZE_FOLLOWER else '❌ Disabled'}")
if RANDOMIZE_FOLLOWER:
    print(f"   Position std range: {FOLLOWER_INIT_POS_STD_RANGE}")
    print(f"   Velocity std range: {FOLLOWER_INIT_VEL_STD_RANGE}")
else:
    print(f"   Fixed position std: 0.5")
    print(f"   Fixed velocity std: 0.2")

# %% [markdown]
# ## 📡 Communication Topology (Simulation Only)
# 
# **注意**：拓扑仅用于**模拟通信范围**，神经网络不使用拓扑结构！

# %%
from topology import CommunicationTopology

topology = CommunicationTopology(NUM_FOLLOWERS, num_pinned=NUM_PINNED)
topology.visualize()

# %% [markdown]
# ## 🔍 Environment Test - Verify Randomization

# %%
from environment import BatchedModelFreeEnv
import torch

# 测试随机初始化
test_env = BatchedModelFreeEnv(topology, num_envs=4)

print("🧪 Testing Randomization (4 parallel environments):")
print("=" * 60)

for test_round in range(3):
    _ = test_env.reset()
    
    print(f"\n📍 Reset #{test_round + 1}:")
    print(f"   Leader Amplitudes: {test_env.leader_amplitude.cpu().numpy().round(3)}")
    print(f"   Leader Omegas: {test_env.leader_omega.cpu().numpy().round(3)}")
    print(f"   Leader Phases: {test_env.leader_phase.cpu().numpy().round(3)}")
    print(f"   Follower Pos Std: {test_env.positions[:, 1:].std(dim=1).cpu().numpy().round(3)}")

# %% [markdown]
# ## 🚀 CTDE Training
# 
# **关键特性：**
# - ✅ 每个 episode 随机化领导者动力学
# - ✅ 每个 episode 随机化跟随者初始状态
# - ✅ Actor 只用本地观测（分散执行）
# - ✅ Critic 用全局状态（集中训练）

# %%
from train import train
from config import NUM_EPISODES

trained_agent, topology, dashboard = train(
    num_episodes=NUM_EPISODES, 
    vis_interval=5,
    show_dashboard=True
)

# %% [markdown]
# ## 📊 Evaluation on Different Scenarios
# 
# 评估智能体在**不同领导者动力学**下的泛化能力

# %%
from utils import plot_evaluation, collect_trajectory, evaluate_agent
from environment import ModelFreeEnv
import matplotlib.pyplot as plt
import numpy as np

# 标准评估
print("📊 Standard Evaluation (Randomized Scenarios):")
print("=" * 50)
plot_evaluation(trained_agent, topology, num_tests=3, save_path='final_evaluation_ctde.png')

# %% [markdown]
# ## 🔬 Generalization Test - Different Leader Dynamics

# %%
# 测试不同领导者动力学下的性能
def test_specific_dynamics(agent, topology, amplitude, omega, num_steps=600):
    """测试特定领导者动力学下的性能"""
    from environment import ModelFreeEnv
    from config import DEVICE
    
    env = ModelFreeEnv(topology)
    
    # 手动设置领导者参数（覆盖随机化）
    env.batched_env.leader_amplitude[0] = amplitude
    env.batched_env.leader_omega[0] = omega
    env.batched_env.leader_phase[0] = 0.0
    
    state = env.reset()
    # 重新设置参数（reset 会重新随机化）
    env.batched_env.leader_amplitude[0] = amplitude
    env.batched_env.leader_omega[0] = omega
    env.batched_env.leader_phase[0] = 0.0
    
    total_reward = 0
    total_error = 0
    total_comm = 0
    
    times = [0]
    leader_pos = [env.positions[0].item()]
    follower_pos = [env.positions[1:].cpu().numpy()]
    
    for step in range(num_steps):
        action = agent.select_action(state, deterministic=True)
        state, reward, _, info = env.step(action)
        
        total_reward += reward
        total_error += info['tracking_error']
        total_comm += info['comm_rate']
        
        times.append(env.t)
        leader_pos.append(env.positions[0].item())
        follower_pos.append(env.positions[1:].cpu().numpy())
    
    return {
        'reward': total_reward,
        'avg_error': total_error / num_steps,
        'avg_comm': total_comm / num_steps,
        'times': np.array(times),
        'leader_pos': np.array(leader_pos),
        'follower_pos': np.array(follower_pos)
    }

# 测试不同场景
test_scenarios = [
    {'name': 'Slow & Small', 'amplitude': 1.0, 'omega': 0.3},
    {'name': 'Medium', 'amplitude': 2.0, 'omega': 0.5},
    {'name': 'Fast & Large', 'amplitude': 3.0, 'omega': 0.8},
    {'name': 'Extra Fast', 'amplitude': 2.5, 'omega': 1.0},  # 超出训练范围
]

print("\n🔬 Generalization Test - Different Leader Dynamics:")
print("=" * 70)
print(f"{'Scenario':<15} | {'Amplitude':>9} | {'Omega':>7} | {'Reward':>10} | {'Avg Error':>10} | {'Comm Rate':>9}")
print("-" * 70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, scenario in enumerate(test_scenarios):
    result = test_specific_dynamics(
        trained_agent, topology, 
        scenario['amplitude'], scenario['omega']
    )
    
    print(f"{scenario['name']:<15} | {scenario['amplitude']:>9.1f} | {scenario['omega']:>7.2f} | "
          f"{result['reward']:>10.2f} | {result['avg_error']:>10.4f} | {result['avg_comm']*100:>8.1f}%")
    
    # 绘图
    ax = axes[idx]
    ax.plot(result['times'], result['leader_pos'], 'r-', lw=2.5, label='Leader')
    
    colors = plt.cm.Blues(np.linspace(0.3, 0.9, result['follower_pos'].shape[1]))
    for i in range(min(4, result['follower_pos'].shape[1])):
        ax.plot(result['times'], result['follower_pos'][:, i], color=colors[i], alpha=0.7, lw=1)
    
    avg_follower = result['follower_pos'].mean(axis=1)
    ax.plot(result['times'], avg_follower, 'g--', lw=2, label='Avg Follower')
    
    ax.set_title(f"{scenario['name']} (A={scenario['amplitude']}, ω={scenario['omega']})\n"
                f"Error: {result['avg_error']:.4f}, Comm: {result['avg_comm']*100:.1f}%", fontsize=10)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('generalization_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("-" * 70)
print("✅ Generalization test complete!")

# %% [markdown]
# ## 📈 CTDE State Space Analysis

# %%
from config import (
    STATE_DIM, LOCAL_OBS_DIM, MAX_NEIGHBORS, NEIGHBOR_OBS_DIM,
    GLOBAL_STATE_DIM, NUM_AGENTS, ACTION_DIM, NUM_FOLLOWERS
)

print("📊 CTDE State Space Analysis:")
print("=" * 60)

print("\n🎭 Actor (Decentralized Execution):")
print(f"   Input: Local observation ({STATE_DIM} dim)")
print(f"     - Self state: {LOCAL_OBS_DIM} dim (position, velocity)")
print(f"     - Neighbor data: {MAX_NEIGHBORS} × {NEIGHBOR_OBS_DIM} = {MAX_NEIGHBORS * NEIGHBOR_OBS_DIM} dim")
print(f"   Output: Action ({ACTION_DIM} dim)")
print(f"     - Velocity change: 1 dim")
print(f"     - Communication threshold: 1 dim")

print("\n🎯 Critic (Centralized Training):")
print(f"   Input: Global state ({GLOBAL_STATE_DIM} dim) + Joint action ({NUM_FOLLOWERS * ACTION_DIM} dim)")
print(f"     - All agents' positions: {NUM_AGENTS} dim")
print(f"     - All agents' velocities: {NUM_AGENTS} dim")
print(f"     - All followers' actions: {NUM_FOLLOWERS} × {ACTION_DIM} = {NUM_FOLLOWERS * ACTION_DIM} dim")
print(f"   Output: Q-value (1 dim)")

print("\n🔑 Key CTDE Properties:")
print("   ✅ Actor does NOT access global state during execution")
print("   ✅ Critic has full observability during training")
print("   ✅ Each agent uses shared policy (parameter sharing)")
print("   ✅ Randomized scenarios improve generalization")

# %% [markdown]
# ## 💾 Save Results

# %%
# 保存训练摘要
if dashboard:
    summary = dashboard.get_summary()
    print("\n📊 Training Summary:")
    print(f"   Best Reward: {summary['best_reward']:.2f}")
    print(f"   Final Reward: {summary['final_reward']:.2f}")
    print(f"   Final Tracking Error: {summary['final_tracking_error']:.4f}")
    print(f"   Final Comm Rate: {summary['final_comm_rate']*100:.1f}%")
    print(f"   Total Episodes: {summary['total_episodes']}")
    print(f"   Elapsed Time: {summary['elapsed_time']:.1f}s")

# 保存模型信息
from config import SAVE_MODEL_PATH
print(f"\n💾 Model saved to: {SAVE_MODEL_PATH}")